# TAREA FINAL: Topología Computacional

Este notebook integra todas las funcionalidades requeridas:
1. **Álgebra Lineal en Z2** (Forma de Smith).
2. **Clase SimplicialComplex** (Estructura, Caras, Euler, Conexas, Betti).
3. **Constructores Geométricos** (Alfa Complejos y Vietoris-Rips).
4. **Verificación** con los datos del PDF proporcionado.

In [ ]:
# BLOQUE 1: IMPORTACIONES Y ÁLGEBRA LINEAL (Z2)

import numpy as np
import matplotlib.pyplot as plt
from itertools import combinations
from scipy.spatial import Delaunay

def smithform(matrix):
    """
    Calcula el rango de una matriz sobre Z_2 usando eliminación Gaussiana.
    Referencia: Clase 9, Forma normal de Smith.
    """
    M = matrix.copy().astype(int)
    rows, cols = M.shape
    rank = 0
    r, c = 0, 0 
    
    while r < rows and c < cols:
        pivot = np.where(M[r:, c:] == 1)
        if pivot[0].size == 0:
            break
        
        pr, pc = pivot[0][0] + r, pivot[1][0] + c
        
        # Swap filas y columnas para traer el pivote a (r,c)
        M[[r, pr], :] = M[[pr, r], :]
        M[:, [c, pc]] = M[:, [pc, c]]
        
        # Eliminar 1s en la fila r y columna c
        for j in range(cols):
            if j != c and M[r, j] == 1:
                M[:, j] = (M[:, j] + M[:, c]) % 2
        
        for i in range(rows):
            if i != r and M[i, c] == 1:
                M[i, :] = (M[i, :] + M[r, :]) % 2
                
        rank += 1
        r += 1
        c += 1
        
    return rank

In [ ]:
# BLOQUE 2: CLASE PRINCIPAL (SimplicialComplex)

class SimplicialComplex:
    """
    Clase para manejar Complejos Simpliciales.
    Cumple los requisitos de: Estructura, Caras, Estrellas, Links, Filtración y Homología.
    """
    def __init__(self, simplices=None):
        self._simplices = set()
        self._simplex_values = {} # Diccionario para almacenar el valor de filtración

        if simplices:
            self.insert(simplices)

    # --- Gestión de Datos y Filtración ---

    def add_simplex(self, simplex, value=0.0):
        """Añade un símplice y todas sus caras, manteniendo la propiedad de clausura."""
        simplex = frozenset(simplex)
        if not simplex: return

        # Añadimos el símplice y todas sus caras (combinations)
        for r in range(1, len(simplex) + 1):
            for face in combinations(simplex, r):
                face = frozenset(face)
                self._simplices.add(face)
                
                # Actualizamos el valor de filtración.
                # Si una cara ya existe (ej. arista compartida), nos quedamos con el valor MENOR
                # para respetar la regla de nacimiento más temprana posible en construcciones como Alpha.
                val_float = float(value)
                if face not in self._simplex_values:
                    self._simplex_values[face] = val_float
                else:
                    self._simplex_values[face] = min(self._simplex_values[face], val_float)

    def insert(self, simplices_input, value=0.0):
        """Interfaz flexible para insertar listas de símplices o tuplas ((simplex), value)."""
        for item in simplices_input:
            # Detecta si es formato ((0,1), 0.5) o solo (0,1)
            if isinstance(item, tuple) and len(item) == 2 and isinstance(item[0], (tuple, list)):
                self.add_simplex(item[0], item[1])
            else:
                self.add_simplex(item, value)

    def filtration(self, t):
        """Retorna un subcomplejo con símplices nacidos antes o en el tiempo t."""
        valid_data = [(s, v) for s, v in self._simplex_values.items() if v <= t]
        return SimplicialComplex(valid_data)
    
    @property
    def sorted_simplices(self):
        """
        Devuelve la lista de símplices ordenados por valor de filtración y luego dimensión.
        Requisito de la imagen 2 y PDF pag 3.
        """
        # Orden: 1. Valor (flotante), 2. Dimensión (len), 3. Lexicográfico (tuple)
        return sorted(
            self._simplex_values.items(), 
            key=lambda x: (x[1], len(x[0]), tuple(sorted(x[0])))
        )

    # --- Propiedades Topológicas Básicas ---

    @property
    def dimension(self):
        if not self._simplices: return -1
        return max(len(f) for f in self._simplices) - 1

    def n_faces(self, n):
        """Devuelve caras de dimensión n ordenadas."""
        faces = [f for f in self._simplices if len(f) - 1 == n]
        return sorted([tuple(sorted(f)) for f in faces])

    @property
    def face_set(self):
        """Todas las caras ordenadas por dimensión y lexicográficamente."""
        return sorted([tuple(sorted(f)) for f in self._simplices], key=lambda x: (len(x), x))

    @property
    def euler_characteristic(self):
        """Suma alternada de símplices: Σ (-1)^k * s_k"""
        return sum((-1)**(len(f)-1) for f in self._simplices)

    def st(self, simplex):
        """Estrella de un símplice: todas las co-caras."""
        target = frozenset(simplex)
        return [tuple(sorted(s)) for s in self._simplices if target.issubset(s)]

    def lk(self, simplex):
        """Link de un símplice: frontera de la estrella."""
        target = frozenset(simplex)
        link_faces = []
        for s in self._simplices:
            if target.issubset(s):
                diff = s - target
                if diff:
                    link_faces.append(tuple(sorted(diff)))
        return SimplicialComplex(link_faces)

    def connected_components(self):
        """Número de componentes conexas usando Union-Find (Betti_0)."""
        vertices = set(v for s in self._simplices for v in s)
        parent = {v: v for v in vertices}

        def find(v):
            if parent[v] != v: parent[v] = find(parent[v])
            return parent[v]

        def union(u, v):
            root_u, root_v = find(u), find(v)
            if root_u != root_v: parent[root_u] = root_v

        for edge in self.n_faces(1):
            union(edge[0], edge[1])
            
        return len({find(v) for v in vertices}) if vertices else 0

    # --- Álgebra Homológica ---

    def boundary_matrix(self, p):
        """Calcula la matriz de borde M_p: C_p -> C_{p-1}."""
        if p <= 0: return np.zeros((0, 0))
        
        row_faces = self.n_faces(p-1) # (p-1)-símplices
        col_faces = self.n_faces(p)   # p-símplices
        
        if not col_faces: return np.zeros((0, 0))
        if not row_faces: return np.zeros((0, len(col_faces))) 

        idx_map = {frozenset(f): i for i, f in enumerate(row_faces)}
        M = np.zeros((len(row_faces), len(col_faces)), dtype=int)

        for j, sigma in enumerate(col_faces):
            sigma_set = set(sigma)
            for v in sigma: 
                face = frozenset(sigma_set - {v})
                if face in idx_map:
                    M[idx_map[face], j] = 1
        return M

    def betti_numbers(self):
        """Calcula Betti usando rangos."""
        dim = self.dimension
        bettis = {}
        ranks = {}

        # Pre-calcular rangos de matrices de borde
        for p in range(1, dim + 2):
            M = self.boundary_matrix(p)
            ranks[p] = smithform(M) if M.size > 0 else 0
        
        # Betti 0
        s0 = len(self.n_faces(0))
        bettis[0] = s0 - ranks.get(1, 0)

        # Betti p > 0
        for p in range(1, dim + 1):
            sp = len(self.n_faces(p))
            bettis[p] = sp - ranks.get(p, 0) - ranks.get(p + 1, 0)
            
        return bettis

In [ ]:
# BLOQUE 3: CONSTRUCTORES GEOMÉTRICOS (Alpha & Rips)

def circumradius(points_simplex):
    """Calcula el circunradio de un símplice (arista o triángulo)."""
    pts = np.array(points_simplex)
    # Caso Arista
    if len(pts) == 2:
        return np.linalg.norm(pts[0] - pts[1]) / 2.0
    # Caso Triángulo
    elif len(pts) == 3:
        a = np.linalg.norm(pts[0] - pts[1])
        b = np.linalg.norm(pts[1] - pts[2])
        c = np.linalg.norm(pts[2] - pts[0])
        s = (a + b + c) / 2.0
        area = np.sqrt(max(0, s * (s - a) * (s - b) * (s - c))) # max(0,...) evita errores numéricos
        if area == 0: return float('inf')
        return (a * b * c) / (4.0 * area)
    return 0.0

def calculate_alpha_complex(points):
    """
    Calcula la filtración del Alfa Complejo.
    Añade todos los símplices de Delaunay con sus circunradios.
    La clase SimplicialComplex manejará la consistencia (minimizando valores).
    """
    tri = Delaunay(points)
    alpha_simplices = []
    
    # 1. Vértices (Radio 0)
    for i in range(len(points)):
        alpha_simplices.append(((i,), 0.0))
        
    # 2. Triángulos y sus caras
    # Nota: Insertamos el triángulo con su radio. 
    # Insertamos las aristas con SU propio radio (L/2).
    # Si una arista no es Gabriel, geométricamente su "tiempo de entrada" en Alpha
    # sería el del triángulo, pero al usar add_simplex en la clase,
    # si insertamos arista con L/2 y luego triángulo con R, 
    # el sistema de filtración debe interpretarse correctamente.
    # Para igualar el PDF, usaremos los valores geométricos directos.
    
    for simplex_indices in tri.simplices:
        idx = tuple(sorted(simplex_indices))
        pts = points[list(idx)]
        r_tri = circumradius(pts)
        alpha_simplices.append((idx, r_tri))
        
        # Aristas del triángulo
        for edge_indices in combinations(idx, 2):
            edge = tuple(sorted(edge_indices))
            pts_edge = points[list(edge)]
            r_edge = circumradius(pts_edge)
            alpha_simplices.append((edge, r_edge))

    return SimplicialComplex(alpha_simplices)

def calculate_rips_manual(points, epsilon, max_dim=2):
    """Construye Vietoris-Rips calculando distancias manualmente."""
    num_points = len(points)
    complex_data = []
    
    # 0-símplices
    for i in range(num_points):
        complex_data.append(((i,), 0.0))
        
    # 1-símplices
    for i in range(num_points):
        for j in range(i + 1, num_points):
            dist = np.linalg.norm(points[i] - points[j])
            if dist <= epsilon:
                complex_data.append(((i, j), dist))
    
    # 2-símplices (Cliques)
    if max_dim >= 2:
        for i in range(num_points):
            for j in range(i+1, num_points):
                for k in range(j+1, num_points):
                     d1 = np.linalg.norm(points[i]-points[j])
                     d2 = np.linalg.norm(points[j]-points[k])
                     d3 = np.linalg.norm(points[i]-points[k])
                     if d1 <= epsilon and d2 <= epsilon and d3 <= epsilon:
                         max_edge = max(d1, d2, d3)
                         complex_data.append(((i,j,k), max_edge))
                         
    return SimplicialComplex(complex_data)

def plot_complex_2d(points, complex_obj, title):
    plt.figure(figsize=(6,6))
    # Dibujar triángulos
    for s in complex_obj.n_faces(2):
        pts = points[list(s)]
        plt.fill(pts[:,0], pts[:,1], 'cyan', alpha=0.2)
    # Dibujar aristas
    for s in complex_obj.n_faces(1):
        p1, p2 = points[s[0]], points[s[1]]
        plt.plot([p1[0], p2[0]], [p1[1], p2[1]], 'b-', alpha=0.5)
    # Dibujar puntos
    plt.plot(points[:,0], points[:,1], 'ko')
    plt.title(title)
    plt.grid(True, linestyle='--', alpha=0.3)
    plt.show()

In [ ]:
# BLOQUE 4: EJECUCIÓN CON DATOS DEL PDF

print("=== CARGANDO DATOS EXACTOS DEL PDF ===")
points = np.array([
    (0.38021546727456423, 0.46419202339598786),
    (0.7951628297672293, 0.49263630135869474),
    (0.566623772375203, 0.038325621649018426), 
    (0.3369306814864865, 0.7103735061134965),
    (0.08272837815822842, 0.2263273314352896),
    (0.5180166301873989, 0.6271769943824689),
    (0.33691411899985035, 0.8402045183219995),
    (0.33244488399729255, 0.4524636520475205),
    (0.11778991601260325, 0.46657734204021165),
    (0.9384303415747769, 0.2313873874340855)
])

# 1. GENERAR ALFA COMPLEJO
print("\n=== ALFA COMPLEJO ===")
K_alpha = calculate_alpha_complex(points)

# VERIFICACIÓN 1: LISTA ORDENADA (PDF PAG 3)
print("\n--- Filtración Ordenada (Primeros 20) ---")
count = 0
for s, val in K_alpha.sorted_simplices:
    print(f"{tuple(sorted(s))}: {val:.4f}")
    count += 1
    if count >= 20: break

# VERIFICACIÓN 2: UMBRALES (PDF PAG 4)
print("\n--- Lista de Umbrales (Primeros 10) ---")
thresholds = sorted(list(set(v for s, v in K_alpha.sorted_simplices)))
print(thresholds[:10])

# VERIFICACIÓN 3: SNAPSHOT EN r=0.26 (PDF PAG 5)
r_snap = 0.26
K_snap = K_alpha.filtration(r_snap)
print(f"\n--- Alfa Complejo filtrado en r={r_snap} ---")
print(f"Total símplices: {len(K_snap.face_set)}")
print(f"Betti: {K_snap.betti_numbers()}")

plot_complex_2d(points, K_snap, f"Alfa Complejo (r={r_snap})")

# 2. VIETORIS-RIPS
print("\n=== VIETORIS-RIPS (r=0.26) ===")
# Nota: Epsilon en Rips suele ser el diámetro (2*r), depende de convención.
# Aquí probamos con el valor directo.
K_vr = calculate_rips_manual(points, epsilon=0.26)
print(f"Betti VR: {K_vr.betti_numbers()}")